In [71]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import math
import imageio
import glob2 as glob
import os
import shutil
from skimage.measure import compare_ssim as ssim

In [56]:
pathToVideos = "/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/"

In [65]:
def getAllDirectories (pathToDirectory):
    '''
        Function that gets all the subdirectories of a folder
        
        Args:
            path: Path to root directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    files = glob.glob(pathToDirectory + '*/', recursive=True)
    return files

def getAllFiles (pathToFiles):
    '''
        Function that gets all the filenames from a directory
        
        Args:
            path: Path to directory
            
        Returns: List of all the complete paths to files
    '''
    files = []
    print(pathToFiles)
    files = glob.glob(pathToFiles + '*/*/', recursive=True)
    return files

In [25]:
directoriesList = getAllDirectories(pathToVideos)

In [26]:
def extractFramesToFile (pathToFile, dest):

    vidcap = cv2.VideoCapture(pathToFile)    
    success = True
    count = 0
    
    while success:
        success,image = vidcap.read()
        if success:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(dest + "frame" + str(count) + ".jpg", gray)     # save frame as JPEG file
            if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                break
            count += 1
    # When everything done, release the capture
    vidcap.release()
    cv2.destroyAllWindows()

In [27]:
# It was used to get frames from video and save them as images
for path in directoriesList:
    innerFiles = glob.glob(path + '/*')
    for j, innerPath in enumerate(innerFiles):
        folderName = innerPath.rsplit('/', 1)[1][:-4]
        writePath = os.path.join(path, folderName + "/")
        os.mkdir(writePath)
        extractFramesToFile(innerPath, writePath)

In [66]:
filesList = getAllFiles(pathToVideos)

/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/


In [68]:
#Initializing variables

TOTAL_KEY_FRAMES = 10

STOPPING_ITERATION = 10

NUMBER_OF_NP_CANDIDATES = 10

# Scale factor. (prev .2) (pretty good 0.6)
F = 0.9

# Cr probability value. (prev .5) (pretty good 0.9)
Cr = 0.6

In [94]:
# Calculate ASSIM for a chromosome.
def getASSIM(KF, filepath):
    ssim_sum = 0
    for i in range(0, TOTAL_KEY_FRAMES - 2):
        while True:
            try:
                im1 = cv2.imread(filepath + "frame" + str(KF[i]) + ".jpg",0)
                im2 = cv2.imread(filepath + "frame" + str(KF[i+1]) + ".jpg",0)
                ssim_sum += ssim(im1, im2)
            except:
#                 print i, KF, KF[i], KF[i+1]
#                 continue
                print (i, KF[i], KF[i+1])
                raise
            break
    return ssim_sum/(TOTAL_KEY_FRAMES - 1)

# INITIALISATION : Generates population NP of 10 parent vectors (and ASSIMs).
def initialize_NP(maxNumberOfFrames, filepath):
    print (maxNumberOfFrames)
    for i in range(NUMBER_OF_NP_CANDIDATES):
        NP.append(sorted(random.sample(range(0, maxNumberOfFrames), TOTAL_KEY_FRAMES)))
        NP[-1].append(getASSIM(NP[-1], filepath))
#         print NP[-1]


# MUTATION
def mutation(parent, maxNumberOfFrames, filepath):
    R = random.sample(NP,2)
    global MV
    MV[:] = []
    MV_value = 0
#     print NP[parent]
    for i in range(TOTAL_KEY_FRAMES):
        MV_value = int(NP[parent][i] + F*(R[0][i] - R[1][i]))
        if(MV_value < 1):
            MV.append(1)
        elif(MV_value > maxNumberOfFrames-1):
            MV.append(maxNumberOfFrames-1)
        else:
            MV.append(MV_value)
    MV.sort()
    MV.append(getASSIM(MV, filepath))

    
# CROSSOVER (uniform crossover with Cr = 0.6).
def crossover(parent, mutant, filepath):
    print ("mutant: ", mutant)
    print ("parent: ", parent)
    for j in range(TOTAL_KEY_FRAMES) :
        if(random.uniform(0,1) < Cr) :
            TV.append(mutant[j])
        else:
            TV.append(parent[j])
    TV.sort()
    TV.append(getASSIM(TV, filepath))

# SELECTION : Selects offspring / parent based on lower ASSIM value.
def selection(parent, trail_vector):
    if(trail_vector[-1] < parent[-1]):
        parent[:] = trail_vector
        print ("yes", parent)
    else:
        print ("no")

# bestParent returns the parent with then minimum ASSIM value.
def bestParent(population):
    Min_ASSIM_value = population[0][-1]
    Best_Parent_Index = population[0]
    for parent in population:
        if (parent[-1] < Min_ASSIM_value):
            Min_ASSIM_value = parent[-1]
            Best_Parent_Index = parent
    return Best_Parent_Index

In [95]:
for i, framesPath in enumerate(filesList):
    # Population matrix.
    NP = []
    # Mutation vector.
    MV = []
    # Trail vector.
    TV = []
    listOfFiles = os.listdir(framesPath) # dir is your directory path
    framesQnt = len(listOfFiles)
    
    print ("Starting routine for file " + framesPath)
    print ("Quantity of frames: " + str(framesQnt))
    
    initialize_NP(framesQnt, framesPath)
    
    for GENERATION in range(STOPPING_ITERATION):
        for i in range(NUMBER_OF_NP_CANDIDATES):
            print ("---------------------", "PARENT:", i+1 , "GENERATION:", GENERATION+1, "---------------------")
            mutation(i, framesQnt, framesPath)
            crossover(NP[i], MV, framesPath)
            selection(NP[i], TV)
            TV[:] = []
    best_parent = bestParent(NP)
    print ("best solution is: ", best_parent)
    
#     images_for_gif = []
    resultPath = os.path.join(framesPath, "DEResult/")
    os.mkdir(resultPath)
    for frame_number in best_parent[:-1]:
        shutil.copyfile(franesPath + "frame" + str(frame_number) + ".jpg", resultPath + "frame" + str(frame_number) + ".jpg")
#         images_for_gif.append(imageio.imread(framesPath + "frame" + str(frame_number) + ".jpg"))
#     imageio.mimsave(framesPath + "result.gif", images_for_gif)

no
--------------------- PARENT: 5 GENERATION: 8 ---------------------
mutant:  [4, 29, 44, 48, 50, 61, 68, 71, 75, 80, 0.8285681861825748]
parent:  [1, 13, 38, 44, 45, 54, 66, 74, 83, 83, 0.823278644094768]
yes [1, 13, 44, 48, 50, 61, 68, 71, 75, 83, 0.822521613308063]
--------------------- PARENT: 6 GENERATION: 8 ---------------------
mutant:  [4, 7, 17, 22, 30, 40, 68, 69, 72, 79, 0.8313574591851522]
parent:  [1, 1, 1, 16, 27, 33, 63, 75, 80, 82, 0.8250639600282205]
no
--------------------- PARENT: 7 GENERATION: 8 ---------------------
mutant:  [1, 1, 1, 10, 24, 26, 55, 79, 83, 83, 0.8320228906326057]
parent:  [0, 1, 17, 21, 35, 40, 44, 61, 68, 80, 0.8135225972070752]
no
--------------------- PARENT: 8 GENERATION: 8 ---------------------
mutant:  [1, 15, 17, 25, 36, 45, 55, 59, 68, 80, 0.8168126632406231]
parent:  [1, 4, 4, 20, 35, 42, 55, 71, 74, 76, 0.8249136810914252]
no
--------------------- PARENT: 9 GENERATION: 8 ---------------------
mutant:  [2, 15, 19, 31, 42, 61, 69, 74, 7